# Information Retrieval

## Creating Documents

The Reuters-21578 files are in [SGML format](http://kdd.ics.uci.edu/databases/reuters21578/README.txt). We need to split them into individual documents. There should be 1000.

In [5]:
import re

In [4]:
test = '''Showers continued throughout the week in
the Bahia cocoa zone, alleviating the drought since early
January and improving prospects for the coming temporao,
although normal humidity levels have not been restored,
Comissaria Smith said in its weekly review.
    The dry period means the temporao will be late this year.
    Arrivals for the week ended February 22 were 155,221 bags
of 60 kilos making a cumulative total for the season of 5.93
mln against 5.81 at the same stage last year. Again it seems
that cocoa delivered earlier on consignment was included in the
arrivals figures.
    Comissaria Smith said there is still some doubt as to how
much old crop cocoa is still available as harvesting has
practically come to an end. With total Bahia crop estimates
around 6.4 mln bags and sales standing at almost 6.2 mln there
are a few hundred thousand bags still in the hands of farmers,
middlemen, exporters and processors.
    There are doubts as to how much of this cocoa would be fit
for export as shippers are now experiencing dificulties in
obtaining +Bahia superior+ certificates.'''

In [6]:
re.split('\W+', test)


['Showers',
 'continued',
 'throughout',
 'the',
 'week',
 'in',
 'the',
 'Bahia',
 'cocoa',
 'zone',
 'alleviating',
 'the',
 'drought',
 'since',
 'early',
 'January',
 'and',
 'improving',
 'prospects',
 'for',
 'the',
 'coming',
 'temporao',
 'although',
 'normal',
 'humidity',
 'levels',
 'have',
 'not',
 'been',
 'restored',
 'Comissaria',
 'Smith',
 'said',
 'in',
 'its',
 'weekly',
 'review',
 'The',
 'dry',
 'period',
 'means',
 'the',
 'temporao',
 'will',
 'be',
 'late',
 'this',
 'year',
 'Arrivals',
 'for',
 'the',
 'week',
 'ended',
 'February',
 '22',
 'were',
 '155',
 '221',
 'bags',
 'of',
 '60',
 'kilos',
 'making',
 'a',
 'cumulative',
 'total',
 'for',
 'the',
 'season',
 'of',
 '5',
 '93',
 'mln',
 'against',
 '5',
 '81',
 'at',
 'the',
 'same',
 'stage',
 'last',
 'year',
 'Again',
 'it',
 'seems',
 'that',
 'cocoa',
 'delivered',
 'earlier',
 'on',
 'consignment',
 'was',
 'included',
 'in',
 'the',
 'arrivals',
 'figures',
 'Comissaria',
 'Smith',
 'said',
 'the